## Overview

`dlt` is an open-source library that you can add to your Python scripts to load data from various and often messy data sources into well-structured, live datasets.

How it works?

`dlt` extracts data from a source, inspects its structure to generate a schema, organizes, normalizes and verifies the data, and loads the data into a destination, such as a database.


![img](../dlt_resources/dlt-high-level.png)

Below, we give you a preview of how you can get data from APIs, files, Python objects or pandas dataframes and move it into a local or remote database, data lake or a vector data store. 

Let's get started!

## Installation

Official releases of dlt can be installed from [PyPI](https://pypi.org/project/dlt/):

In [ ]:
!pip install -q dlt

Command above just installs library core, in example below we use `duckdb` as a [destination](https://dlthub.com/docs/dlt-ecosystem/destinations), so let's add it:

In [ ]:
!pip install -q "dlt[duckdb]"

> Use clean virtual environment for your experiments! Here are [detailed instructions](https://dlthub.com/docs/reference/installation).

## Quick start

Let's load a list of Python objects (dicts) into `duckdb` database and inspect the created dataset.

> We gonna use `full_refresh` for our test examples. If you create a new pipeline script you will be experimenting a lot. If you want that each time the pipeline resets its state and loads data to a new dataset, set the full_refresh argument of the dlt.pipeline method to True. Each time the pipeline is created, dlt adds datetime-based suffix to the dataset name.

In [1]:
import dlt

data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='quick_start',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=True, 
)
load_info = pipeline.run(data, table_name="users")
print(load_info)

Pipeline quick_start load step completed in 0.41 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20240210113244
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/quick_start.duckdb location to store data
Load package 1707564764.3130832 is LOADED and contains no failed jobs


### Now explore your data! 

To see the schema of your created database, run Streamlit command:

```python
 dlt pipeline <pipeline_name> show
```
[This command](https://dlthub.com/docs/reference/command-line-interface#show-tables-and-data-in-the-destination) generates and launches a simple Streamlit app that you can use to inspect the schemas and data in the destination.

To use `streamlit`, install it first.

For example above pipeline name is “quick_start”, so run:

In [ ]:
# !pip install -q streamlit pandas==2.0.0

In [2]:
!dlt pipeline quick_start show

Found pipeline quick_start in /home/ellabelle/.dlt/pipelines

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.17.156.62:8501

^C
  Stopping...


## Load data from variety of sources

Use dlt to load practically any data you deal with in your Python script into a dataset. 

The library will create/update tables, infer data types and deal with nested data automatically:
- list of dicts
- json
- csv
- API
- database
- etc.

### from JSON

When creating a schema during normalization, dlt recursively unpacks this nested structure into relational tables, creating and linking [children and parent tables](https://dlthub.com/docs/dlt-ecosystem/visualizations/understanding-the-tables#child-and-parent-tables).

In [ ]:
# create test json file

import json

with open("test.json", 'w') as file:
    data = {
        'id': 1, 
        'name': 'Alice', 
        'job': {
            "company": "ScaleVector",
            "title": "Data Scientist",
        },
        'children': [
            {
                'id': 1, 
                'name': 'Eve'
            },
            {
                'id': 2, 
                'name': 'Wendy'
            }
        ]
    }
    json.dump(data, file)


In [ ]:
# load test json to duckdb database

import json
import dlt


with open("test.json", 'r') as file:
    data = json.load(file)


pipeline = dlt.pipeline(
	pipeline_name='from_json',
	destination='duckdb', 
	dataset_name='mydata',
    full_refresh=True,
)
# dlt works with lists of dicts, so wrap data to the list
load_info = pipeline.run([data], table_name="json_data")
print(load_info)

In [ ]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM json_data").df()
data_table

### from API

Below we load 100 most recent issues from our [own dlt repository](https://github.com/dlt-hub/dlt) into "issues" table.

In [3]:
import dlt
import requests


# url to request dlt-hub/dlt issues
url = "https://api.github.com/repos/dlt-hub/dlt/issues"
# make the request and check if succeeded
response = requests.get(url)
response.raise_for_status()

pipeline = dlt.pipeline(
	pipeline_name='from_api',
	destination='duckdb', 
	dataset_name='mydata',
    full_refresh=True,
)
load_info = pipeline.run(response.json(), table_name="issues")
print(load_info)

Pipeline from_api load step completed in 0.50 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata_20240210113549
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/from_api.duckdb location to store data
Load package 1707564949.1378694 is LOADED and contains no failed jobs


In [5]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT COUNT(*) FROM issues").df()
data_table.head()

┌──────────┬──────────────────────┬─────────────────────┬──────────────────────┬───────────────────────────┬───────────┐
│ database │        schema        │        name         │     column_names     │       column_types        │ temporary │
│ varchar  │       varchar        │       varchar       │      varchar[]       │         varchar[]         │  boolean  │
├──────────┼──────────────────────┼─────────────────────┼──────────────────────┼───────────────────────────┼───────────┤
│ from_api │ mydata_20240210113…  │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGI…  │ false     │
│ from_api │ mydata_20240210113…  │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHA…  │ false     │
│ from_api │ mydata_20240210113…  │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMEST…  │ false     │
│ from_api │ mydata_20240210113…  │ issues              │ [url, repository_u…  │ [VARCHAR, VARCHAR, VARC…  │ false     │
│ from_api │ mydata_20240210113…

,count_star()
0,30


## Append or replace your data

Run this examples twice and you notice that each time a copy of the data is added to your tables.
We call this load mode `append`. It is very useful when i.e. you have a new folder created daily with `json` file logs, and you want to ingest them.

In [8]:
import dlt


data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='append',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(data, table_name="users")
print(load_info)

Pipeline append load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/append.duckdb location to store data
Load package 1707565088.623157 is LOADED and contains no failed jobs


In [9]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table

,id,name,_dlt_load_id,_dlt_id
0,1,Alice,1707565077.6717787,fuMOCw9PM1dGww
1,2,Bob,1707565077.6717787,yNjPgpXAk956uw
2,1,Alice,1707565088.623157,ODQKJh/RDwqRUg
3,2,Bob,1707565088.623157,CZSY1xsmg4hhvg


Perhaps this is not what you want to do in the example above.
For example, if the CSV file is updated, how we can refresh it in the database?
One method is to tell `dlt` to replace the data in existing tables by using `write_disposition`.

In [10]:
import dlt


data = [
	{'id': 1, 'name': 'Alice'},
	{'id': 2, 'name': 'Bob'}
]

pipeline = dlt.pipeline(
	pipeline_name='replace',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(data, table_name="users", write_disposition="replace")
print(load_info)

Pipeline replace load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/replace.duckdb location to store data
Load package 1707565129.7864115 is LOADED and contains no failed jobs


In [11]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table

,id,name,_dlt_load_id,_dlt_id
0,1,Alice,1707565129.7864115,DSCVEqoUN2KkdA
1,2,Bob,1707565129.7864115,IpwvkBzTBPz4gQ


## Declare loading behavior

You can finetune the loading process by decorating Python functions with `@dlt.resource`.

### Load only new data (incremental loading)

We can supercharge the example above and get only users that were created since last load.
Instead of using `replace` write_disposition and downloading all users each time the pipeline is run, we do the following:

In [16]:
import dlt


data = [
	{'id': 1, 'name': 'Alice', 'created_at': "2023-09-01"},
	{'id': 2, 'name': 'Bob', 'created_at': "2023-09-02"},
    {'id': 3, 'name': 'Chad', 'created_at': "2023-09-03"},
    {'id': 4, 'name': 'Carol', 'created_at': "2023-09-04"}
]

@dlt.resource
def users(
    created_at=dlt.sources.incremental("created_at", initial_value="2023-08-01")
):
    yield from data
    
pipeline = dlt.pipeline(
	pipeline_name='incremental',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(users)
print(load_info)

Pipeline incremental load step completed in 0.37 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/incremental.duckdb location to store data
Load package 1707571366.9300675 is LOADED and contains no failed jobs


We use the `@dlt.resource` decorator to declare table name to which data will be loaded and write disposition, which is `append` by default.

We also use `dlt.sources.incremental` to track `created_at` field present in each user to filter only the newly created ones.

Now run the script. It loads all the users from our test data to `duckdb`. Run it again, and you can see that no users got added.

In [17]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table.head()

,id,name,created_at,_dlt_load_id,_dlt_id
0,1,Alice,2023-09-01,1707571366.9300675,EfYj9z8D3FECFw
1,2,Bob,2023-09-02,1707571366.9300675,pHAxCj2yDI4NWQ
2,3,Chad,2023-09-03,1707571366.9300675,jklX34Tm3oWh6g
3,4,Carol,2023-09-04,1707571366.9300675,U+FXAr5VjHYpjw


## Update and deduplicate your data

The script above finds new users and adds them to the database.
It will ignore any updates to user information.
Get always fresh content of all the users: combine an incremental load with `merge` write disposition,
like in the script below.

In [18]:
import dlt


data = [
	{'id': 1, 'name': 'Alice', 'created_at': "2023-09-01", 'updated_at': "2023-09-01"},
	{'id': 2, 'name': 'Boba', 'created_at': "2023-09-02", 'updated_at': "2023-09-05"},
    {'id': 3, 'name': 'Chad', 'created_at': "2023-09-03", 'updated_at': "2023-09-03"},
    {'id': 4, 'name': 'Carol', 'created_at': "2023-09-04", 'updated_at': "2023-09-04"}
]

@dlt.resource(
    write_disposition="merge",
    primary_key="id",
)
def users(
    updated_at=dlt.sources.incremental("updated_at", initial_value="2023-08-01")
):
    yield from data
    
pipeline = dlt.pipeline(
	pipeline_name='merge',
	destination='duckdb',
	dataset_name='mydata',
    full_refresh=False, 
)
load_info = pipeline.run(users)
print(load_info)

Pipeline merge load step completed in 0.67 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/merge.duckdb location to store data
Load package 1707571404.8401496 is LOADED and contains no failed jobs


Above we add `primary_key` hint that tells `dlt` how to identify the users in the database to find duplicates which content it will merge.


In [19]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM users").df()
data_table.head()

,id,name,created_at,updated_at,_dlt_load_id,_dlt_id
0,4,Carol,2023-09-04,2023-09-04,1707571404.8401496,HVFQgPOBENAxAQ
1,1,Alice,2023-09-01,2023-09-01,1707571404.8401496,MvxcK4MGB6BziQ
2,2,Boba,2023-09-02,2023-09-05,1707571404.8401496,eiwUPKwI2eRmLQ
3,3,Chad,2023-09-03,2023-09-03,1707571404.8401496,TQiBo0jrGu1Enw


## Real life example

We can improve the GitHub API example above and get only issues that were created since last load.

In [20]:
import dlt
import requests


@dlt.resource(
    table_name="issues",
    write_disposition="merge",
    primary_key="id",
)
def get_issues(
    updated_at = dlt.sources.incremental("updated_at", initial_value="1970-01-01T00:00:00Z")
):
    # url to request dlt-hub issues
    url = f"https://api.github.com/repos/dlt-hub/dlt/issues?since={updated_at.last_value}"

    while True:
        response = requests.get(url)
        page_items = response.json()

        if len(page_items) == 0:
            break
        yield page_items

        if "next" not in response.links:
            break
        url = response.links["next"]["url"]


pipeline = dlt.pipeline(
    pipeline_name='github_issues_merge',
    destination='duckdb',
    dataset_name='mydata',
    full_refresh=False,
)
# dlt works with lists of dicts, so wrap data to the list
load_info = pipeline.run(get_issues)
print(load_info)

Pipeline github_issues_merge load step completed in 1.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset mydata
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/github_issues_merge.duckdb location to store data
Load package 1707571416.876398 is LOADED and contains no failed jobs



Note that we now track the `updated_at` field - so we filter in all issues **updated** since the last pipeline run (which also includes newly created ones).

Also pay attention how we use **since** [GitHub API](https://docs.github.com/en/rest/issues/issues?apiVersion=2022-11-28#list-repository-issues)
and `updated_at.last_value` to tell GitHub which issues we are interested in. `updated_at.last_value` holds the last `updated_at` value from the previous run.

Now you can run this script on a daily schedule, and each day you'll load only issues created after the time of the previous pipeline run.

In [21]:
import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
data_table = conn.sql("SELECT * FROM issues").df()
data_table.head()

/tmp/ipykernel_67847/2972541876.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_table = conn.sql("SELECT * FROM issues").df()


,id,url,repository_url,labels_url,comments_url,events_url,html_url,node_id,number,title,...,performed_via_github_app__permissions__actions,performed_via_github_app__permissions__checks,performed_via_github_app__permissions__contents,performed_via_github_app__permissions__deployments,performed_via_github_app__permissions__discussions,performed_via_github_app__permissions__issues,performed_via_github_app__permissions__metadata,performed_via_github_app__permissions__pull_requests,performed_via_github_app__permissions__repository_projects,performed_via_github_app__permissions__statuses
0,2126291188,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/954,PR_kwDOGvRYu85mb7da,954,Add git to filesystem source 301,...,None,None,None,None,None,None,None,None,None,None
1,2126159467,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/953,PR_kwDOGvRYu85mbfKI,953,passes incremental from apply hints to resourc...,...,None,None,None,None,None,None,None,None,None,None
2,2126134270,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/952,PR_kwDOGvRYu85mbZvc,952,855 create bigquery adapter for dlt resources,...,None,None,None,None,None,None,None,None,None,None
3,2125791707,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/951,PR_kwDOGvRYu85maOZo,951,Handle UnionType when checking is_union_type a...,...,None,None,None,None,None,None,None,None,None,None
4,2125375604,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://api.github.com/repos/dlt-hub/dlt/issue...,https://github.com/dlt-hub/dlt/pull/950,PR_kwDOGvRYu85mYxul,950,"Tweaked output to not include ""Found schema"" a...",...,None,None,None,None,None,None,None,None,None,None


### Use existed verified sources

To use existed verified source, just run the `dlt init` [command](https://dlthub.com/docs/reference/command-line-interface#dlt-init).

List all verified sources:

In [12]:
!dlt init --list-verified-sources

Looking up for verified sources in https://github.com/dlt-hub/verified-sources.git...
pipedrive: Highly customizable source for Pipedrive, supports endpoint addition, selection and column rename
notion: A source that extracts data from Notion API
salesforce: Source for Salesforce depending on the simple_salesforce python package.
hubspot: This is a module that provides a DLT source to retrieve data from multiple endpoints of the HubSpot API using a specified API key. The retrieved data is returned as a tuple of Dlt resources, one for each endpoint.
github: Source that load github issues, pull requests and reactions for a specific repository via customizable graphql query. Loads events incrementally.
chess: A source loading player profiles and games from chess.com api
stripe_analytics: This source uses Stripe API and dlt to load data such as Customer, Subscription, Event etc. to the database and to calculate the MRR and churn rate. 
shopify_dlt: Fetches Shopify Orders and Products.
zend

This command shows all available verified sources and their short descriptions. For each source, checks if your local `dlt` version requires update and prints the relevant warning.

Consider an example of a pipeline for Pokemon API.

This command will initialize the pipeline example with Pokemon as the source and `duckdb` as the [destination](https://dlthub.com/docs/dlt-ecosystem/destinations):


In [13]:
!dlt --non-interactive init pokemon duckdb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source pokemon (This source provides data extraction from an example source as a starting point for new pipelines.)

Verified source pokemon was added to your project!
* See the usage examples and code snippets to copy from pokemon_pipeline.py
* Add credentials for duckdb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information


In [14]:
!python pokemon_pipeline.py

Pipeline pokemon load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset pokemon_data
The duckdb destination used duckdb:////mnt/c/Users/ellabelle/Github/data-engineering-zoomcamp/cohorts/2024/workshops/dlt_resources/pokemon.duckdb location to store data
Load package 1707571327.2131693 is LOADED and contains no failed jobs


In [15]:
import duckdb

conn = duckdb.connect(f"pokemon.duckdb")
conn.sql(f"SET search_path = 'pokemon_data'")
display(conn.sql("DESCRIBE"))
data_table = conn.sql("SELECT * FROM pokemon").df()
data_table

┌──────────┬──────────────┬─────────────────────┬──────────────────────┬───────────────────────────────────┬───────────┐
│ database │    schema    │        name         │     column_names     │           column_types            │ temporary │
│ varchar  │   varchar    │       varchar       │      varchar[]       │             varchar[]             │  boolean  │
├──────────┼──────────────┼─────────────────────┼──────────────────────┼───────────────────────────────────┼───────────┤
│ pokemon  │ pokemon_data │ _dlt_loads          │ [load_id, schema_n…  │ [VARCHAR, VARCHAR, BIGINT, TIME…  │ false     │
│ pokemon  │ pokemon_data │ _dlt_pipeline_state │ [version, engine_v…  │ [BIGINT, BIGINT, VARCHAR, VARCH…  │ false     │
│ pokemon  │ pokemon_data │ _dlt_version        │ [version, engine_v…  │ [BIGINT, BIGINT, TIMESTAMP WITH…  │ false     │
│ pokemon  │ pokemon_data │ berries             │ [name, url, _dlt_l…  │ [VARCHAR, VARCHAR, VARCHAR, VAR…  │ false     │
│ pokemon  │ pokemon_data │ poke

,name,url,_dlt_load_id,_dlt_id
0,bulbasaur,https://pokeapi.co/api/v2/pokemon/1/,1707571327.2131693,kRcAPvqYNKoAxg
1,ivysaur,https://pokeapi.co/api/v2/pokemon/2/,1707571327.2131693,PzSkfn5ScYOu4g
2,venusaur,https://pokeapi.co/api/v2/pokemon/3/,1707571327.2131693,AN5lJasTDa5HTw
3,charmander,https://pokeapi.co/api/v2/pokemon/4/,1707571327.2131693,C36D/mh95FyE2g
4,charmeleon,https://pokeapi.co/api/v2/pokemon/5/,1707571327.2131693,pcSsexwQBy6qrw
5,charizard,https://pokeapi.co/api/v2/pokemon/6/,1707571327.2131693,Sy3UEvw0OUVWqg
6,squirtle,https://pokeapi.co/api/v2/pokemon/7/,1707571327.2131693,cIRaNmP5o+6log
7,wartortle,https://pokeapi.co/api/v2/pokemon/8/,1707571327.2131693,WriOrmND60uVow
8,blastoise,https://pokeapi.co/api/v2/pokemon/9/,1707571327.2131693,4PiuBk0tYuHCEw
9,caterpie,https://pokeapi.co/api/v2/pokemon/10/,1707571327.2131693,7ul6Wt4IjW/uaA
